# Traffic analysis for fleet, trucks and equipment

A transportation / taxi company has collected massive historical traffic data and wants to find traffic patterns, predict congestions, and suggest optimized routes for its own vehicles.

To this end, we can join traffic records with points of interest or geographic zones to identify the hot spots and congestion points.

We recommend to use a Sedona cluster with 40+ CPU cores and 100GB+ RAM for running this notebook. 

## Import necessary dependencies

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit, concat, aggregate, size, count, desc, struct, avg, year, month, dayofmonth

from sedona.spark import *
from IPython.display import display, HTML
import json

## Create SedonaContext


In [2]:
builder = SedonaContext.builder().\
    config("spark.driver.maxResultSize", "10g"). \
    config("sedona.global.indextype", "rtree"). \
    config("spark.wherobots.raster.outdb.readahead", "4m")

sedona = SedonaContext.create(builder.getOrCreate())

23/10/22 09:04:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/22 09:04:06 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/10/22 09:04:07 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to pvoaenkhn3/spark-logs/spark-b3540f6251524196a67cfb23b21212ce.inprogress. This is unsupported


## Explore the Wherobots open data catalog

The following commands show the `wherobots_pro_data` catalog of Wherobots open data. This will print the databases and tables inside this catalog.

In [3]:
sedona.sql("SHOW SCHEMAS IN wherobots_pro_data").show()
sedona.sql("SHOW TABLES IN wherobots_pro_data.google_microsoft").show()
sedona.sql("SHOW TABLES IN wherobots_pro_data.us_census").show()
sedona.sql("SHOW TABLES IN wherobots_pro_data.nyc_taxi").show()

+----------------+
|       namespace|
+----------------+
|google_microsoft|
|         landsat|
|        nyc_taxi|
|             osm|
|       us_census|
|         weather|
+----------------+



+----------------+--------------+-----------+
|       namespace|     tableName|isTemporary|
+----------------+--------------+-----------+
|google_microsoft|open_buildings|      false|
+----------------+--------------+-----------+



+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|us_census|  zipcode|      false|
+---------+---------+-----------+

+---------+----------------+-----------+
|namespace|       tableName|isTemporary|
+---------+----------------+-----------+
| nyc_taxi|yellow_2009_2010|      false|
+---------+----------------+-----------+



## Understand Table open_buildings

The dataset integrates Google's V3 Open Buildings with the newest Building Footprints from Microsoft. Wherobots takes this combined dataset from VIDA and converts it to Havasu format with spatial index. This dataset contains over 2.5 billion building footprints.

For demo purpose, we only take the Microsoft-version building footprints in New York City.

Its schema and count are as follows.

Then we use SedonaKepler to visualize a sample of the data to understand its data distribution.

In [4]:
buildings = sedona.table("wherobots_pro_data.google_microsoft.open_buildings")
buildings.printSchema()
buildings.show()
buildings = buildings.filter("ST_Intersects(ST_PolygonFromEnvelope(-74.2591, 40.4774, -73.7004, 40.9176), geometry)").filter("bf_source = 'microsoft'").repartition(400)
print("NYC buildings count: ", buildings.count())
buildings.createOrReplaceTempView("buildings_nyc")
SedonaKepler.create_map(buildings.sample(0.1), "buildings")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


root
 |-- geometry: geometry (nullable = true)
 |-- boundary_id: long (nullable = true)
 |-- bf_source: string (nullable = true)
 |-- confidence: double (nullable = true)
 |-- area_in_meters: double (nullable = true)
 |-- country_iso: string (nullable = true)



+--------------------+-----------+---------+----------+------------------+-----------+
|            geometry|boundary_id|bf_source|confidence|    area_in_meters|country_iso|
+--------------------+-----------+---------+----------+------------------+-----------+
|POLYGON ((29.0227...|        312|microsoft|      null| 73.11497087016033|        UKR|
|POLYGON ((29.0240...|        312|microsoft|      null|103.20214841906869|        UKR|
|POLYGON ((29.0147...|        312|microsoft|      null| 33.33722540112282|        UKR|
|POLYGON ((29.0137...|        312|microsoft|      null|21.408372270656038|        UKR|
|POLYGON ((29.0116...|        312|microsoft|      null| 55.15879787922788|        UKR|
|POLYGON ((29.0148...|        312|microsoft|      null| 21.57436274439953|        UKR|
|POLYGON ((29.0166...|        312|microsoft|      null|18.720202300347005|        UKR|
|POLYGON ((29.0078...|        312|microsoft|      null| 45.98465251154108|        UKR|
|POLYGON ((29.0208...|        312|microsoft

NYC buildings count:  980808
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/opt/conda/lib/python3.10/site-packages/jupyter_client/session.py:719: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'buildings': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…

## Understand Table yellow_2009_2010

Trip records from yellow taxis were gathered and supplied to the NYC Taxi and Limousine Commission (TLC). These records detail pick-up and drop-off times, locations, journey lengths, fare breakdowns, rate categories, modes of payment, and the number of passengers as reported by drivers.

We only use data from Year 2009 and 2010 as new data provided NYC TLC no longer includes exact geospatial locations.

Its schema and count are as follows.

Then we use SedonaKepler to visualize a sample of the data to understand its data distribution.

In [5]:
taxis = sedona.table("wherobots_pro_data.nyc_taxi.yellow_2009_2010")
taxis.printSchema()
# Remove outlier data
taxis = taxis.filter("ST_Intersects(ST_PolygonFromEnvelope(-74.2591, 40.4774, -73.7004, 40.9176), pickup_location)").filter("ST_Intersects(ST_PolygonFromEnvelope(-74.2591, 40.4774, -73.7004, 40.9176), dropoff_location)")
print("Taxi records count: ", taxis.count())
taxis.createOrReplaceTempView("taxis")
SedonaKepler.create_map(taxis.selectExpr("pickup_location", "passenger_count", "vendor_id").sample(0.001), "taxis")

root
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- pickup_location: geometry (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- dropoff_location: geometry (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)



Taxi records count:  182852811
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'taxis': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

## Understand Table zipcode (ZCTA)

ZCTAs, short for ZIP Code Tabulation Areas represent a geographical entity developed by the U.S. Census Bureau. Their purpose is to facilitate visualization, mapping, and spatial analysis of the dataset associated with the USPS's Zone Improvement Plan, commonly known as ZIP Codes.

Its schema and count are as follows.

Then we use SedonaKepler to visualize a sample of the data to understand its data distribution.

In [6]:
zipcodes = sedona.table("wherobots_pro_data.us_census.zipcode")
zipcodes.printSchema()
# Only keep zones in NYC
zipcodes = zipcodes.filter("ST_Intersects(ST_PolygonFromEnvelope(-74.2591, 40.4774, -73.7004, 40.9176), geometry)")
print("Zip codes count: " , zipcodes.count())
zipcodes.createOrReplaceTempView("zones_nyc")
SedonaKepler.create_map(zipcodes.selectExpr("geometry", "ZCTA5CE10"), "zones")

root
 |-- geometry: geometry (nullable = true)
 |-- ZCTA5CE10: string (nullable = true)
 |-- GEOID10: string (nullable = true)
 |-- CLASSFP10: string (nullable = true)
 |-- MTFCC10: string (nullable = true)
 |-- FUNCSTAT10: string (nullable = true)
 |-- ALAND10: decimal(14,0) (nullable = true)
 |-- AWATER10: decimal(14,0) (nullable = true)
 |-- INTPTLAT10: string (nullable = true)
 |-- INTPTLON10: string (nullable = true)



Zip codes count:  349
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'zones': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

## Find the most popular zones for taxi pickups

This requires 2 steps:

1. Perform a spatial join between Table zipcode and Table yellow_2009_2010 to find taxi pickup locations in each zone.
2. Perform an aggregation query on the join result to count the number of pickup locations per zones, and sort the result descendingly.

One observation is that Manhattan region is the most popular area in the New York City.

In [7]:
pickup_zone = sedona.sql(
    """
    SELECT 
        z.geometry as zipgeom, 
        z.ZCTA5CE10 as zipcode, 
        t.total_amount 
    FROM 
        zones_nyc z, 
        taxis t 
    WHERE 
        ST_Intersects(z.geometry, t.pickup_location) 
    """
)
pickup_zone = pickup_zone.groupBy("zipcode", "zipgeom").agg(count("*").alias("count")).orderBy(desc("count"))
pickup_zone = pickup_zone.cache()
pickup_zone.show()

# Provide a config file that can produce heatmap directly. You can also manually config a heap map via the KeplerGL panel
map_config = json.loads('{"version": "v1", "config": {"visState": {"filters": [], "layers": [{"id": "292htir", "type": "geojson", "config": {"dataId": "pickup_zone", "label": "pickup_zone", "color": [246, 209, 138], "highlightColor": [252, 242, 26, 255], "columns": {"geojson": "geometry"}, "isVisible": true, "visConfig": {"opacity": 0.8, "strokeOpacity": 0.8, "thickness": 0.5, "strokeColor": [183, 136, 94], "colorRange": {"name": "Global Warming", "type": "sequential", "category": "Uber", "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"]}, "strokeColorRange": {"name": "Global Warming", "type": "sequential", "category": "Uber", "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"]}, "radius": 10, "sizeRange": [0, 10], "radiusRange": [0, 50], "heightRange": [0, 500], "elevationScale": 5, "enableElevationZoomFactor": true, "stroked": true, "filled": true, "enable3d": false, "wireframe": false}, "hidden": false, "textLabel": [{"field": null, "color": [255, 255, 255], "size": 18, "offset": [0, 0], "anchor": "start", "alignment": "center"}]}, "visualChannels": {"colorField": {"name": "count", "type": "integer"}, "colorScale": "quantile", "strokeColorField": null, "strokeColorScale": "quantile", "sizeField": null, "sizeScale": "linear", "heightField": null, "heightScale": "linear", "radiusField": null, "radiusScale": "linear"}}], "interactionConfig": {"tooltip": {"fieldsToShow": {"pickup_zone": [{"name": "zipcode", "format": null}, {"name": "count", "format": null}]}, "compareMode": false, "compareType": "absolute", "enabled": true}, "brush": {"size": 0.5, "enabled": false}, "geocoder": {"enabled": false}, "coordinate": {"enabled": false}}, "layerBlending": "normal", "splitMaps": [], "animationConfig": {"currentTime": null, "speed": 1}}, "mapState": {"bearing": 0, "dragRotate": false, "latitude": 40.754194999999996, "longitude": -73.963679, "pitch": 0, "zoom": 9, "isSplit": false}, "mapStyle": {"styleType": "dark", "topLayerGroups": {}, "visibleLayerGroups": {"label": true, "road": true, "border": false, "building": true, "water": true, "land": true, "3d building": false}, "threeDBuildingColor": [9.665468314072013, 17.18305478057247, 31.1442867897876], "mapStyles": {}}}}')
SedonaKepler.create_map(pickup_zone, "pickup_zone", config = map_config)

+-------+--------------------+--------+
|zipcode|             zipgeom|   count|
+-------+--------------------+--------+
|  10019|POLYGON ((-74.003...|12130600|
|  10003|POLYGON ((-73.999...|10628842|
|  10022|POLYGON ((-73.977...|10185182|
|  10016|POLYGON ((-73.987...| 9727619|
|  10011|POLYGON ((-74.012...| 9494464|
|  10036|POLYGON ((-74.001...| 8843262|
|  10001|POLYGON ((-74.008...| 8538118|
|  10017|POLYGON ((-73.981...| 6952021|
|  10023|POLYGON ((-73.990...| 6918065|
|  10014|POLYGON ((-74.015...| 6587465|
|  10021|MULTIPOLYGON (((-...| 6157016|
|  10065|POLYGON ((-73.972...| 6001119|
|  10012|POLYGON ((-74.004...| 5537702|
|  10010|POLYGON ((-73.993...| 5499812|
|  10024|POLYGON ((-73.988...| 5185363|
|  10128|MULTIPOLYGON (((-...| 5182653|
|  10028|POLYGON ((-73.963...| 4731805|
|  10013|POLYGON ((-74.016...| 4637368|
|  10025|POLYGON ((-73.977...| 4035095|
|  10018|POLYGON ((-74.006...| 3968276|
+-------+--------------------+--------+
only showing top 20 rows

User Guide: ht

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '292htir', 'type': …

## Find the most expensive zones for taxi

We want to find the average trip fare for taxis picked up in each zone. This might help policy makers design taxi pricing models.

This requires 2 steps:

1. Perform a spatial join between Table zipcode and Table yellow_2009_2010 to find taxi pickup locations in each zone.
2. Perform an aggregation query on the join result to average the trip fare of trip records per zones, and sort the result descendingly.

One observation is that people in Manhattan region usually do short-distance trip while people from airports usually pay more.

In [8]:
fare_zone = sedona.sql(
    """
    SELECT 
        z.geometry as zipgeom, 
        z.ZCTA5CE10 as zipcode, 
        t.total_amount 
    FROM 
        zones_nyc z, 
        taxis t 
    WHERE 
        ST_Intersects(z.geometry, t.pickup_location) 
    """
)
fare_zone = fare_zone.groupBy("zipcode", "zipgeom").agg(avg("total_amount").alias("fare")).orderBy(desc("fare"))
fare_zone = fare_zone.cache()
fare_zone.show()

# Provide a config file that can produce heatmap directly. You can also manually config a heap map via the KeplerGL panel
map_config = json.loads('{"version": "v1", "config": {"visState": {"filters": [], "layers": [{"id": "g5xd43b", "type": "geojson", "config": {"dataId": "fare_zone", "label": "fare_zone", "color": [30, 150, 190], "highlightColor": [252, 242, 26, 255], "columns": {"geojson": "geometry"}, "isVisible": true, "visConfig": {"opacity": 0.8, "strokeOpacity": 0.8, "thickness": 0.5, "strokeColor": [137, 218, 193], "colorRange": {"name": "Global Warming", "type": "sequential", "category": "Uber", "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"]}, "strokeColorRange": {"name": "Global Warming", "type": "sequential", "category": "Uber", "colors": ["#5A1846", "#900C3F", "#C70039", "#E3611C", "#F1920E", "#FFC300"]}, "radius": 10, "sizeRange": [0, 10], "radiusRange": [0, 50], "heightRange": [0, 500], "elevationScale": 5, "enableElevationZoomFactor": true, "stroked": true, "filled": true, "enable3d": false, "wireframe": false}, "hidden": false, "textLabel": [{"field": null, "color": [255, 255, 255], "size": 18, "offset": [0, 0], "anchor": "start", "alignment": "center"}]}, "visualChannels": {"colorField": {"name": "fare", "type": "real"}, "colorScale": "quantile", "strokeColorField": null, "strokeColorScale": "quantile", "sizeField": null, "sizeScale": "linear", "heightField": null, "heightScale": "linear", "radiusField": null, "radiusScale": "linear"}}], "interactionConfig": {"tooltip": {"fieldsToShow": {"fare_zone": [{"name": "zipcode", "format": null}, {"name": "fare", "format": null}]}, "compareMode": false, "compareType": "absolute", "enabled": true}, "brush": {"size": 0.5, "enabled": false}, "geocoder": {"enabled": false}, "coordinate": {"enabled": false}}, "layerBlending": "normal", "splitMaps": [], "animationConfig": {"currentTime": null, "speed": 1}}, "mapState": {"bearing": 0, "dragRotate": false, "latitude": 40.754194999999996, "longitude": -73.963679, "pitch": 0, "zoom": 9, "isSplit": false}, "mapStyle": {"styleType": "dark", "topLayerGroups": {}, "visibleLayerGroups": {"label": true, "road": true, "border": false, "building": true, "water": true, "land": true, "3d building": false}, "threeDBuildingColor": [9.665468314072013, 17.18305478057247, 31.1442867897876], "mapStyles": {}}}}')
fare_zone_map = SedonaKepler.create_map(fare_zone, "fare_zone", config = map_config)
fare_zone_map

+-------+--------------------+------------------+
|zipcode|             zipgeom|              fare|
+-------+--------------------+------------------+
|  10538|POLYGON ((-73.772...| 65.88181818181818|
|  07114|POLYGON ((-74.212...| 60.27721553466339|
|  07470|POLYGON ((-74.289...| 59.79456692913387|
|  10801|POLYGON ((-73.805...|57.352236135957085|
|  07102|POLYGON ((-74.185...|56.864615384615355|
|  07043|POLYGON ((-74.218...| 54.51392344497612|
|  07311|POLYGON ((-74.034...| 47.79757451181915|
|  07201|POLYGON ((-74.217...| 47.01816506567343|
|  11561|POLYGON ((-73.706...| 46.60166666666667|
|  11557|POLYGON ((-73.714...| 46.10167832167832|
|  07079|POLYGON ((-74.283...|41.892181818181825|
|  11042|POLYGON ((-73.712...| 40.95830769230769|
|  11430|POLYGON ((-73.832...| 40.57335020023982|
|  11030|POLYGON ((-73.712...| 39.72695804195804|
|  11436|POLYGON ((-73.805...|39.668798383384896|
|  07505|MULTIPOLYGON (((-...|39.285714285714285|
|  10805|MULTIPOLYGON (((-...| 38.70259398496241|


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'g5xd43b', 'type': …

## Find the busiest buildings in New York City

We want to know which buildings lead to lots of taxi pickups and their surrounding areas might be congestion points.

This requires 2 steps:

1. Perform a distance join between Table open_buildings and Table yellow_2009_2010 to find taxis picked up within 100 meter distance of each building.
2. Perform an aggregation query on the join result to count the pick up records per building, and sort the result descendingly.

One observation is that the busiest building is the one at the intersection of W 31st St and 7th Ave, which is Madison Square Garden, known as The Garden or in initials as MSG. It is a multi-purpose indoor arena in New York City. Located in Midtown Manhattan between 7th and 8th Avenues from 31st to 33rd Streets, it is situated atop Pennsylvania Station. People should avoid this congestion point when driving.

In [9]:
taxi_buildings = sedona.sql(
    """
    SELECT
        t.total_amount,
        b.geometry as buildinggeom, 
        b.boundary_id as boundary_id
    FROM 
        taxis t,
        buildings_nyc b
    WHERE 
        ST_DistanceSphere(t.pickup_location, b.geometry) <= 30
    """
)
taxi_buildings = taxi_buildings.groupBy("boundary_id", "buildinggeom").agg(count("total_amount").alias("count")).orderBy(desc("count"))
taxi_buildings = taxi_buildings.cache()
taxi_buildings.show(10)
taxi_buildings_map = SedonaKepler.create_map(taxi_buildings.limit(10), "taxi_buildings")
taxi_buildings_map

23/10/22 09:06:55 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+-----------+--------------------+------+
|boundary_id|        buildinggeom| count|
+-----------+--------------------+------+
|        116|POLYGON ((-73.992...|204201|
|        116|POLYGON ((-73.992...|164735|
|        116|POLYGON ((-73.991...|158489|
|        116|POLYGON ((-74.005...|148474|
|        116|POLYGON ((-73.987...|104894|
|        116|POLYGON ((-74.015...|104150|
|        116|POLYGON ((-74.007...|101665|
|        116|POLYGON ((-74.007...| 85147|
|        116|POLYGON ((-73.978...| 83453|
|        116|POLYGON ((-74.002...| 83254|
+-----------+--------------------+------+
only showing top 10 rows

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'taxi_buildings': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'columns': ['boundary_id', 'geometr…